# PoC2 - ValueFromLog

prompt.md の指示を Jupyter Notebook 化しています。

## 0. セットアップ

- SweetViz が必要です。未導入の場合は `pip install sweetviz` を実行してください。
- データは `data/raw_excel` 配下から読み込みます。

In [ ]:
from pathlib import Path
import pandas as pd

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)

DATA_DIR = Path('data/raw_excel')
TARGETS_PATH = DATA_DIR / '対象者リスト.csv'
STATE_PATH = DATA_DIR / '機器状態変化履歴_抽出結果.csv'
REMOTE_PATH = DATA_DIR / '機器遠隔操作履歴_抽出結果.csv'

try:
    import sweetviz as sv
    SWEETVIZ_AVAILABLE = True
except ModuleNotFoundError:
    SWEETVIZ_AVAILABLE = False
    print('SweetViz が未インストールです。`pip install sweetviz` を実行してから SweetViz 用セルを再実行してください。')


## 1. 対象者リストのレコード数

契約者 ID (ContractId) の件数を確認します。

In [ ]:
targets_df = pd.read_csv(TARGETS_PATH, encoding='cp932')
targets_count = len(targets_df)

display(targets_df.head())
print(f'対象者リストのレコード数: {targets_count}')
print(f'ContractId のユニーク件数: {targets_df["ContractId"].nunique()}')


## 2. 機器状態変化履歴データの概要

SweetViz 実行前に pandas でデータ構成を把握します。

In [ ]:
state_df = pd.read_csv(
    STATE_PATH,
    encoding='utf-8-sig',
    parse_dates=['ReportedDate'],
    infer_datetime_format=True
)

display(state_df.head())
display(state_df.describe(include='all').transpose())

print('MessageName の出現上位:')
print(state_df['MessageName'].value_counts().head())
print()
print('EquipmentTypeId の出現上位:')
print(state_df['EquipmentTypeId'].value_counts().head())


## 3. 機器遠隔操作履歴データの概要

pandas で構造を確認します。

In [ ]:
remote_df = pd.read_csv(
    REMOTE_PATH,
    encoding='utf-8-sig',
    parse_dates=['OrderReceiptDate'],
    infer_datetime_format=True
)

display(remote_df.head())
display(remote_df.describe(include='all').transpose())

print('PropertyName の出現上位:')
print(remote_df['PropertyName'].value_counts().head())
print()
print('EquipmentName の出現上位:')
print(remote_df['EquipmentName'].value_counts().head())


## 4. SweetViz レポート (機器状態変化)

`reports/sweetviz_state.html` に出力します。

In [ ]:
if not SWEETVIZ_AVAILABLE:
    raise ModuleNotFoundError('SweetViz が利用できません。`pip install sweetviz` を実行後に再試行してください。')

reports_dir = Path('reports')
reports_dir.mkdir(exist_ok=True)

state_report = sv.analyze(state_df)
state_html_path = reports_dir / 'sweetviz_state.html'
state_report.show_html(filepath=str(state_html_path), open_browser=False)
state_report.show_notebook()


## 5. SweetViz レポート (機器遠隔操作)

`reports/sweetviz_remote.html` に出力します。

In [ ]:
if not SWEETVIZ_AVAILABLE:
    raise ModuleNotFoundError('SweetViz が利用できません。`pip install sweetviz` を実行後に再試行してください。')

reports_dir = Path('reports')
reports_dir.mkdir(exist_ok=True)

remote_report = sv.analyze(remote_df)
remote_html_path = reports_dir / 'sweetviz_remote.html'
remote_report.show_html(filepath=str(remote_html_path), open_browser=False)
remote_report.show_notebook()


In [ ]:
from pathlib import Path
from IPython.display import display
import pandas as pd

REMOTE_PATH = Path("data/raw_excel/機器遠隔操作履歴_抽出結果.csv")
STATE_PATH = Path("data/raw_excel/機器状態変化履歴_抽出結果.csv")
TIME_TOLERANCE = pd.Timedelta(minutes=5)

remote_df = (
    pd.read_csv(REMOTE_PATH, encoding="utf-8-sig")
    .rename(
        columns={
            "PropertyCode": "PropertyCode_remote",
            "PropertyName": "PropertyName_remote",
            "PropertyValue": "PropertyValue_remote",
        }
    )
)
state_df = (
    pd.read_csv(STATE_PATH, encoding="utf-8-sig")
    .rename(
        columns={
            "PropertyCode1": "PropertyCode_state",
            "PropertyName1": "PropertyName_state",
            "PropertyValue1": "PropertyValue_state",
        }
    )
)

# Normalize datetimes/content for comparison
for df in (remote_df, state_df):
    df.replace("NULL", pd.NA, inplace=True)
remote_df["OrderReceiptDate"] = pd.to_datetime(remote_df["OrderReceiptDate"], utc=True)
state_df["ReportedDate"] = pd.to_datetime(state_df["ReportedDate"], utc=True)
remote_df["remote_row_id"] = remote_df.index
state_df["state_row_id"] = state_df.index

join_keys = ["ContractId", "FloorCode", "RoomName", "EquipmentTypeId", "EquipmentName"]

comparison_candidates = remote_df.merge(
    state_df,
    on=join_keys,
    how="inner",
    suffixes=("_remote", "_state"),
)

comparison_candidates["time_diff"] = (
    comparison_candidates["ReportedDate"] - comparison_candidates["OrderReceiptDate"]
).abs()

matches = comparison_candidates.loc[
    comparison_candidates["time_diff"] <= TIME_TOLERANCE
].copy()

matches["content_match"] = (
    matches["PropertyCode_remote"].fillna("").str.casefold()
    == matches["PropertyCode_state"].fillna("").str.casefold()
) & (
    matches["PropertyValue_remote"].fillna("").str.casefold()
    == matches["PropertyValue_state"].fillna("").str.casefold()
)

nearest_state = (
    matches.sort_values(["remote_row_id", "time_diff"])
    .drop_duplicates("remote_row_id", keep="first")
    .rename(
        columns={
            "ReportedDate": "matched_state_time",
            "PropertyCode_state": "matched_property_code",
            "PropertyValue_state": "matched_property_value",
            "MessageName": "matched_message_name",
        }
    )
)

remote_summary = (
    remote_df.merge(
        nearest_state[
            [
                "remote_row_id",
                "state_row_id",
                "matched_state_time",
                "matched_message_name",
                "matched_property_code",
                "matched_property_value",
                "time_diff",
                "content_match",
            ]
        ],
        on="remote_row_id",
        how="left",
    )
    .assign(
        has_state_match=lambda df: df["state_row_id"].notna(),
        time_diff_seconds=lambda df: df["time_diff"].dt.total_seconds(),
    )
)

print(
    remote_summary[
        [
            "remote_row_id",
            "ContractId",
            "OrderReceiptDate",
            "PropertyCode_remote",
            "PropertyValue_remote",
            "matched_state_time",
            "matched_property_code",
            "matched_property_value",
            "time_diff_seconds",
            "content_match",
            "has_state_match",
        ]
    ].head()
)
print(
    f"{remote_summary['has_state_match'].sum()} / {len(remote_summary)} remote logs "
    f"have a state log within ±{TIME_TOLERANCE}."
)

def inspect_remote(remote_row_id: int, limit: int = 5) -> pd.DataFrame:
    """Quickly list the closest state_report rows for a single remote_report."""
    return (
        comparison_candidates.loc[
            comparison_candidates["remote_row_id"] == remote_row_id
        ]
        .sort_values("time_diff")
        .head(limit)[
            [
                "remote_row_id",
                "state_row_id",
                "OrderReceiptDate",
                "ReportedDate",
                "time_diff",
                "PropertyCode_remote",
                "PropertyValue_remote",
                "PropertyCode_state",
                "PropertyValue_state",
            ]
        ]
    )

display(inspect_remote(0))


## 6. remote_report と state_report の比較

remote_report (remote_df) と state_report (state_df) のログを契約/設置情報で突き合わせ、+/-5 分以内で内容が似たイベントを探します。`TIME_TOLERANCE` を変えると許容時間を調整できます。`inspect_remote()` で個別の remote_report 行に紐づく state_report 候補を確認できます。

In [ ]:
remote_df

In [ ]:
TIME_TOLERANCE = pd.Timedelta(minutes=5)
key_cols = ["ContractId", "FloorCode", "RoomName", "EquipmentTypeId", "EquipmentName"]

remote_subset = (
    remote_df.reset_index(drop=True)
    .assign(remote_row_id=lambda df: df.index)
    .loc[:, key_cols + ["remote_row_id", "OrderReceiptDate", "PropertyCode", "PropertyName", "PropertyValue"]]
    .replace("NULL", pd.NA)
    .rename(
        columns={
            "OrderReceiptDate": "remote_time",
            "PropertyCode": "PropertyCode_remote",
            "PropertyName": "PropertyName_remote",
            "PropertyValue": "PropertyValue_remote",
        }
    )
)

state_subset = (
    state_df.reset_index(drop=True)
    .assign(state_row_id=lambda df: df.index)
    .loc[:, key_cols + ["state_row_id", "ReportedDate", "MessageName", "PropertyCode1", "PropertyName1", "PropertyValue1"]]
    .replace("NULL", pd.NA)
    .rename(
        columns={
            "ReportedDate": "state_time",
            "MessageName": "MessageName_state",
            "PropertyCode1": "PropertyCode_state",
            "PropertyName1": "PropertyName_state",
            "PropertyValue1": "PropertyValue_state",
        }
    )
)

comparison_candidates = (
    remote_subset.merge(
        state_subset,
        on=key_cols,
        how="inner",
        suffixes=("_remote", "_state"),
    ).assign(time_diff=lambda df: (df["state_time"] - df["remote_time"]).abs())
)

matches = comparison_candidates.loc[comparison_candidates["time_diff"] <= TIME_TOLERANCE].copy()
matches["content_match"] = (
    matches["PropertyCode_remote"].fillna("").str.casefold()
    == matches["PropertyCode_state"].fillna("").str.casefold()
) & (
    matches["PropertyValue_remote"].fillna("").str.casefold()
    == matches["PropertyValue_state"].fillna("").str.casefold()
)

nearest_state = (
    matches.sort_values(["remote_row_id", "time_diff"])
    .drop_duplicates("remote_row_id", keep="first")
    .rename(
        columns={
            "state_time": "matched_state_time",
            "MessageName_state": "matched_message_name",
            "PropertyCode_state": "matched_property_code",
            "PropertyValue_state": "matched_property_value",
        }
    )
)

remote_state_summary = (
    remote_subset.merge(
        nearest_state[
            [
                "remote_row_id",
                "state_row_id",
                "matched_state_time",
                "matched_message_name",
                "matched_property_code",
                "matched_property_value",
                "time_diff",
                "content_match",
            ]
        ],
        on="remote_row_id",
        how="left",
    )
    .assign(
        has_state_match=lambda df: df["state_row_id"].notna(),
        time_diff_seconds=lambda df: df["time_diff"].dt.total_seconds(),
    )
)

match_count = remote_state_summary["has_state_match"].sum()
total_remote = len(remote_state_summary)
print(
    f"{match_count} / {total_remote} remote_report rows have a state_report within +/-{TIME_TOLERANCE}."
)

display(
    remote_state_summary[
        [
            "remote_row_id",
            "ContractId",
            "remote_time",
            "PropertyCode_remote",
            "PropertyValue_remote",
            "matched_state_time",
            "matched_property_code",
            "matched_property_value",
            "time_diff_seconds",
            "content_match",
        ]
    ].head(10)
)

def inspect_remote(remote_row_id: int, limit: int = 5) -> pd.DataFrame:
    """List the closest state_report entries for a single remote_report row."""
    return (
        comparison_candidates.loc[comparison_candidates["remote_row_id"] == remote_row_id]
        .sort_values("time_diff")
        .head(limit)[
            [
                "remote_row_id",
                "state_row_id",
                "remote_time",
                "state_time",
                "time_diff",
                "PropertyCode_remote",
                "PropertyValue_remote",
                "PropertyCode_state",
                "PropertyValue_state",
            ]
        ]
    )

display(inspect_remote(0))